## Dash App Creation Code

### Written by: Evan Judge, Joyce Hahn, Shannan Alston 

### Set-Up

In [1]:
#Import packages

#OS
import os

##Dash
import dash
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table, register_page, ALL
import dash_bootstrap_components as dbc

##Plotly
import plotly.express as px
import plotly.graph_objects as go

##Pandas
import pandas as pd

##NumPy
import numpy as np

#Requests
import requests

#Below are mostly modules for map:
import fiona 
import geopandas as gpd
import json
import folium
from folium.plugins import Geocoder
import branca.colormap as cm

import PIL
from PIL import Image
from io import BytesIO

#MatPlotLib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl

import base64

### Read-in and Understand Data

In [2]:
import os
import json
import pandas as pd
import numpy as np

# Load configuration from config.json in the repository root.
with open("config.json", "r") as config_file:
    config = json.load(config_file)
    
# Set the base directory using the value from the configuration file.
base_dir = config["base_dir"]

merged_data_file_path = os.path.join(base_dir, "Data", "Final Data", "Merged", "merged_data.csv")

# Dictionary for data types
dtype_dict = {
    "GEO_NAME": str,
    "GEO_ID": str,
    "YEAR": str,
    "AREA_TYPE": str,
    "STATE_FIPS": str,
    "COUNTY_FIPS": str,
    "TRACT_FIPS": str,
    "TRIBAL_FIPS": str,
    "JUD FTE PAYROLL FLAG": str,
    "JUD PTE FLAG": str,
    "JUD PT PAYROLL FLAG": str,
    "JUD INDV UNIT ID": str, 
    "PO FTE FLAG": str,
    "PO FTE PAYROLL FLAG": str,
    "PO INDV UNIT ID": str,
    "RISK_RATNG": str,
    "RESL_RATNG": str,
    "ERQK_EXPT": str, 
    "HWAV_EXPT": str, 
    "HRCN_EXPT": str, 
    "TRND_EXPT": str, 
    "WFIR_EXPT": str
}

# Read in the data CSV using the constructed file path.
df = pd.read_csv(merged_data_file_path, dtype=dtype_dict, low_memory=False)

# Replace specific values with NaN (per ACS estimate and annotation guidelines).
values_to_replace = [-666666666, -999999999, -888888888, -222222222, -333333333, -555555555, "*", "null"]

# Create a copy of the dataframe for further processing.
df_update = df.copy(deep=True)
df_update.replace(values_to_replace, np.nan, inplace=True)

# Print the columns in the data CSV.
print("The columns in the data CSV are:", df.columns.to_list())


C:\Users\Colin\AppData\Local\Temp\ipykernel_19836\3368894233.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_update.replace(values_to_replace, np.nan, inplace=True)


The columns in the data CSV are: ['GEO_NAME', 'TOTAL_POPULATION', 'MEDIAN_AGE', 'PERCENT_NON_HISPANIC_WHITE_ALONE', 'PERCENT_NON_HISPANIC_BLACK_ALONE', 'PERCENT_NON_HISPANIC_AIAN_ALONE', 'PERCENT_NON_HISPANIC_ASIAN_ALONE', 'PERCENT_NON_HISPANIC_NHPI_ALONE', 'PERCENT_NON_HISPANIC_SOR_ALONE', 'PERCENT_NON_HISPANIC_MULTI_ALONE', 'PERCENT_HISPANIC', 'PERCENT_BACHELORS_DEGREE_OR_HIGHER', 'PERCENT_HIGH_SCHOOL_GRADUATE_OR_HIGHER', 'MEDIAN_HOUSEHOLD_INCOME', 'POVERTY_RATE', 'PERCENT_EMPLOYED', 'PERCENT_UNEMPLOYED', 'PERCENT_SPEAK_ENGLISH_LESS_THAN_VERY_WELL', 'PERCENT_CIVILIAN_VETERAN', 'PERCENT_NO_HEALTH_INSURANCE', 'GEO_ID', 'YEAR', 'AREA_TYPE', 'STATE_FIPS', 'COUNTY_FIPS', 'TRACT_FIPS', 'TOTAL_HOUSEHOLDS', 'MARRIED_COUPLED_HOUSEHOLDS', 'COHAB_COUPLED_HOUSEHOLDS', 'MALE_HH_NOSPOUSEPARTNER', 'FEMALE_HH_NOSPOUSEPARTNER', 'AVG_HH_SIZE', 'PERCENT_FOREIGN_BORN', 'PERCENT_MORGAGE_GT_35%_INC', 'PERCENT_UNDER_5', 'PERCENT_5_TO_9', 'PERCENT_10_TO_14', 'PERCENT_15_TO_19', 'PERCENT_20_TO_24', 'PERCENT_

### Read-in and Understand Metadata

In [3]:
# Build the full path to the metadata CSV file.
metadata_file_path = os.path.join(base_dir, "Data", "Final Data", "Metadata", "metadata.csv")

# Read in the metadata CSV with the specified encoding.
metadata_full = pd.read_csv(metadata_file_path, encoding='cp1252')

# Print all the columns in the metadata CSV.
print("The columns in the metadata CSV are:", metadata_full.columns)

The columns in the metadata CSV are: Index(['Variable Name', 'YLABEL', 'Card', 'Group', 'Decimal', 'Per_Dollars',
       'Domain1', 'Domain2', 'Variable_Label', 'Variable_Description',
       'Interpretation', 'Source', 'Data_Description', 'Data_Provenance',
       'Data_link', 'Metadata_link', 'Coverage', 'Frequency', 'Unit',
       'Geography', 'Uncertainty', 'Citation'],
      dtype='object')


In [4]:
# Create a smaller metadata DataFrame with selected variables.
metadata = pd.read_csv(
    metadata_file_path,
    usecols=['Variable Name', 'Card', 'Group', 'Domain1', 'Domain2', 'Variable_Label',
             'Variable_Description', 'Interpretation', 'Source'],
    encoding='cp1252'
)

# Understand the metadata better by printing selected details.
print("The domains in the metadata are:", metadata['Domain1'].unique())
print()

print("The variable groups in the metadata are:", metadata['Group'].unique())
print()

print("The variables in the metadata are:")
print(metadata['Variable Name'])
print()

print("Examples of variable labels in the metadata are:")
print(metadata['Variable_Label'][0:3])
print()

print("Examples of variable descriptions in the metadata are:")
print(metadata['Variable_Description'][0:3])
print()

print("Examples of variable interpretations in the metadata are:")
print(metadata['Interpretation'][0:3])
print()

The domains in the metadata are: ['General' 'Health & Nutrition'
 'Infrastructure, Housing, & Transportation'
 'Income, Poverty, & Social Services' 'Education' 'Public Safety'
 'Disaster Prevention and Relief' 'Environmnet & Natural Resources'
 'Economic Vitality']

The variable groups in the metadata are: ['Population' 'Median age' 'Age' 'Race and Hispanic origin'
 'Educational attainment' 'Income' 'Poverty' 'Employment' 'Language'
 'Veteran' 'Health insurance' 'Households' 'Living arrangements'
 'Household size' 'Foreign-born' 'Housing affordability' 'Children'
 'Disability' 'Food stamps' 'Fertility' 'Health insurance type'
 'Disability type' 'Rural' 'Hospitals' 'Food insecurity' 'Food access'
 'Health status' 'Housing units' 'Home value' 'Rent' 'Commute type'
 'Commute time' 'Occupants' 'Heating type' 'Construction' 'Vacancy'
 'Internet type' 'Household income' 'Social security' 'Public assistance'
 'Family poverty' 'Veteran poverty' 'Educational enrollment'
 'Field of degree' 'Scho

In [5]:
#Create a metadata data frame for the cards
metadata_cards=metadata[metadata['Card']==1]
print(metadata_cards[0:3])

      Variable Name  Card       Group  Domain1 Domain2    Variable_Label  \
0  TOTAL_POPULATION     1  Population  General     NaN  Total Population   
1        MEDIAN_AGE     1  Median age  General     NaN        Median Age   
4   PERCENT_UNDER_5     1         Age  General     NaN   Percent under 5   

                      Variable_Description  \
0         Estimate of the total population   
1                       Median age (years)   
4  Percent of the population under 5 years   

                                      Interpretation  \
0                    The total number of all persons   
1  The age of the middle person, when all persons...   
4  The percent of all persons that are under 5 years   

                            Source  
0  American Community Survey (ACS)  
1  American Community Survey (ACS)  
4  American Community Survey (ACS)  


In [6]:
#Create a metadata data frame for the accordions
metadata_accordions=metadata_cards[['Group','Domain1']].copy().drop_duplicates()
print(metadata_accordions)

                        Group            Domain1
0                  Population            General
1                  Median age            General
4                         Age            General
22   Race and Hispanic origin            General
30     Educational attainment            General
..                        ...                ...
191                  Business  Economic Vitality
193                   Balance  Economic Vitality
194                   Exports  Economic Vitality
195                   Imports  Economic Vitality
196                       GDP  Economic Vitality

[76 rows x 2 columns]


##### Create lists for map dropdowns

In [7]:
years_list = df_update['YEAR'].unique().tolist()
#Removing 2023 & 2024 since most data is not yet available for these year to avoid confusion
years_list.remove('2024')
years_list.remove('2023')

area_type_list = {'region':'Region', 
                  'division':'Division',
                  'state':'State',
                  'county':'County',
                  'tribal_area':'Tribal Area',
                  'tract':'Census Tract'}

def get_domain_list(domain,group):
    dict = {}
    filtered = metadata_full[(metadata_full['Domain1'] == domain)&(metadata_full['Group'] == group)]
    for index,row in filtered.iterrows():
        dict[row['Variable Name']] = row['Variable_Label']
    return(dict)

title_list = metadata_full['Group'].unique().tolist()

#dictionary of states and fips codes, some of the US territories should possibly be removed, since I don't think there are maps
#or data
fips_state_list = {
 '01': 'Alabama',
 '02': 'Alaska',
 '04': 'Arizona',
 '05': 'Arkansas',
 '06': 'California',
 '08': 'Colorado',
 '09': 'Connecticut',
 '10': 'Delaware',
 '11': 'District of Columbia',
 '12': 'Florida',
 '13': 'Georgia',
 '15': 'Hawaii',
 '16': 'Idaho',
 '17': 'Illinois',
 '18': 'Indiana',
 '19': 'Iowa',
 '20': 'Kansas',
 '21': 'Kentucky',
 '22': 'Louisiana',
 '23': 'Maine',
 '24': 'Maryland',
 '25': 'Massachusetts',
 '26': 'Michigan',
 '27': 'Minnesota',
 '28': 'Mississippi',
 '29': 'Missouri',
 '30': 'Montana',
 '31': 'Nebraska',
 '32': 'Nevada',
 '33': 'New Hampshire',
 '34': 'New Jersey',
 '35': 'New Mexico',
 '36': 'New York',
 '37': 'North Carolina',
 '38': 'North Dakota',
 '39': 'Ohio',
 '40': 'Oklahoma',
 '41': 'Oregon',
 '42': 'Pennsylvania',
 '44': 'Rhode Island',
 '45': 'South Carolina',
 '46': 'South Dakota',
 '47': 'Tennessee',
 '48': 'Texas',
 '49': 'Utah',
 '50': 'Vermont',
 '51': 'Virginia',
 '53': 'Washington',
 '54': 'West Virginia',
 '55': 'Wisconsin',
 '56': 'Wyoming',
 '60': 'American Samoa',
 '66': 'Guam',
 '69': 'Northern Mariana Islands',
 '72': 'Puerto Rico',
 '78': 'Virgin Islands'}

### Functions

#### Mapping functions

Shapefiles coming from here, being converted into GEOJSON, use Create_GeoJson.ipynb if updates are needed to these files
or if any updates are made to merged data table

https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html .

Notes:

-This map only works with numeric data, would need to adjust create a different tool for categorical data

-Tract level data is too large for this method of map creation, as such I have it broken up by state, this method could also be adjusted to be by county or something along those lines. I think a more advanced solution would be to host data on a geoserver and call in that data to avoid memory issues.


In [8]:
#Function to assign measures to appropriate format based on metadata, factors are number of decimals and if it should have a
#dollar or percent sign added
def format_value(value, decimal_places, is_percent=""):
    if isinstance(value, str) == True:
        formatted_value = value
        return formatted_value
    elif is_percent == 'P':
        formatted_value = "{:.{}f}%".format(value, decimal_places)
        return formatted_value
    elif is_percent == 'D':
        formatted_value = "${:,.{}f}".format(value, decimal_places)
        return formatted_value
    elif isinstance(value, (int, float)) == True:
        formatted_value = "{:,.{}f}".format(value, decimal_places)
        return formatted_value
    else:
        formatted_value = value
        return formatted_value

#To create dataframes to get neededed quantiles
#input is area_type and overall data frame loaded above. 
#This will calculate 6 quantiles for whatever level of geography as an output data frame
#this is also used to generate the colorscale in other functions
def get_quantiles(dataframe,area_type):
    
    #I think there was an easier way to do this, using table option of quantile function, but I already wrote this and it works
    quantile_df = dataframe[dataframe['AREA_TYPE'] == area_type]

    low_value = quantile_df.quantile(0, numeric_only=True)
    first_quantile = quantile_df.quantile(0.1666, numeric_only=True)
    second_quantile = quantile_df.quantile(0.3333, numeric_only=True)
    third_quantile = quantile_df.quantile(0.5, numeric_only=True)
    fourth_quantile = quantile_df.quantile(0.6666, numeric_only=True)
    fifth_quantile = quantile_df.quantile(0.8333, numeric_only=True)
    high_value = quantile_df.quantile(1, numeric_only=True)

    quantile_df = pd.concat([low_value,first_quantile,second_quantile,third_quantile,fourth_quantile,fifth_quantile,high_value],axis=1)
    quantile_df = quantile_df.set_axis(['low_value', 'first_quantile', 'second_quantile', 'third_quantile', 'fourth_quantile','fifth_quantile','high_value'],axis=1)
    
    return quantile_df

#creates the colormap for a specific measure used in follium
def get_colormap_dict(quantile_df,measure): 
    list = ['low_value', 'first_quantile', 'second_quantile', 'third_quantile', 'fourth_quantile','fifth_quantile','high_value']
    colormap_dict = {}
    for i in list:
        colormap_dict[i] = quantile_df.loc[measure,i]
    
    return colormap_dict
    
#filters dataframe by year
def filter_df_by_year(df, year):
    df_filtered = df[df['YEAR'] == year]
    return df_filtered

#to assign a metadata dataframe
def assign_metadata(input_csv):
    metadata_df = pd.read_csv(input_csv,encoding='cp1252')
    return metadata_df

#create dictionary to help map variable name to label
def create_var_labels_dict(metadata_df):
    var_labels_dict = {}
    for index, row in metadata_df.iterrows():
        var_labels_dict[row['Variable Name']] = row['Variable_Label']
    return var_labels_dict

#create dictionary to help map variable name to description
def create_var_discription_dict(metadata_df):
    var_discription_dict = {}
    for index, row in metadata_df.iterrows():
        var_discription_dict[row['Variable Name']] = row['Variable_Description']
    return var_discription_dict
    
#Creates the scale using branca colormap.
def create_scale(metadata_df,color_map_dict,measure):
    labels_dict =create_var_labels_dict(metadata_df)
    output_colormap = cm.StepColormap(["#ffffcc","#c7e9b4", "#7fcdbb", "#41b6c4","#2c7fb8","#253494"],
                       index= [color_map_dict["low_value"],
                               color_map_dict["first_quantile"],
                               color_map_dict["second_quantile"],
                               color_map_dict["third_quantile"],
                               color_map_dict["fourth_quantile"],
                               color_map_dict["fifth_quantile"],
                               color_map_dict["high_value"]
                              ],
                       vmin=color_map_dict['low_value'],
                       vmax=color_map_dict['high_value'],
                       caption=labels_dict[measure]) 
    return output_colormap

#state is optional argument
#returns the needed json file based on year and area type; used in mapping to for maps to read geojson
def assign_json(year,area_type,state=False):
    path_key = {'county':'county','division':'division','region':'region','state':'state','tract':'tract','tribal_area':'aiannh'}
    
    if area_type == 'tract':
        path = '/data/discover/Data/Geographic Data/'+year+'/state_files_tract/cb_'+year+'_us_'+path_key[area_type]+'_500k_'+state+'.json'
    else:
        path = '/data/discover/Data/Geographic Data/'+year+'/cb_'+year+'_us_'+path_key[area_type]+'_500k.json'
        
    with open(path) as f:
        geojson = json.load(f)
        return geojson
        # Below part, If features need to be assigned then can use, may be for later to help zoom in on certain feature by 
        #default but I don't think this is in use in current code
        #features = geojson['features']
        
#this function assigns the colormap to the selcted varibale, NAs will be assigned the color gray.
#require inputs are feature which is assigned in the create map function, the branca colormap created after finding quantile
#values, the variable that will have values assigned
def style_function(feature,colormap,measure):
    value = feature['properties'][measure]
    if value is not None:
        fill_color = colormap(value)  # Convert color value to string
    else:
        fill_color = 'gray'
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }


#This function creates the scale which is actually displayed using matplotlib instead of branca as the branca had issues with
#overlapping text that was hard to resolve across a large number of possible maps that could be generated
def create_matplot_scale(metadata_df, color_map_dict, measure):
    labels_dict = create_var_labels_dict(metadata_df)
    description_dict = create_var_discription_dict(metadata_df)
    
    # Define colormap
    colors = ["#ffffcc","#c7e9b4", "#7fcdbb", "#41b6c4","#2c7fb8","#253494"]
    fig, ax = plt.subplots(figsize=(11, 1), layout='constrained')
    cmap_name = 'my_custom_colormap'
    cmap = LinearSegmentedColormap.from_list(cmap_name, colors)
    bounds = [color_map_dict["low_value"],
              color_map_dict["first_quantile"],
              color_map_dict["second_quantile"],
              color_map_dict["third_quantile"],
              color_map_dict["fourth_quantile"],
              color_map_dict["fifth_quantile"],
              color_map_dict["high_value"]
             ]
    
    rounded_bounds = []
    
    #For formatting input
    decimal = (metadata_full.loc[metadata_full['Variable Name'] == measure, 'Decimal'].values[0])
    percent_dollar = (metadata_full.loc[metadata_full['Variable Name'] == measure, 'Per_Dollars'].values[0])
    
    for i in bounds:
        value = format_value(i,decimal,percent_dollar)
        rounded_bounds.append(value)

    
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    
    # Plot colorbar
    cbar = fig.colorbar(
        mpl.cm.ScalarMappable(cmap=cmap, norm=norm),
        cax=ax, orientation='horizontal',
        extend='both',
        spacing='uniform',
        label=labels_dict[measure] +":  " +description_dict[measure]
    )
    
    # Manually set tick labels
    cbar.set_ticks(bounds)
    cbar.ax.set_xticklabels(rounded_bounds)
    
     # Encode figure to base64 directly
    buffered = BytesIO()
    fig.savefig(buffered, format="png")
    encoded_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
    plt.close(fig)  # Close the figure to free up memory
    
    return html.Img(src=f"data:image/png;base64,{encoded_image}", style={"height":"150%","width": "50%"})

In [9]:
def create_map(year,area_type,measure,state=False):
    #filter data to year needed
    df_filtered = filter_df_by_year(df_update,year)
    quantile_df = get_quantiles(df_filtered,area_type)
    colormap_dict = get_colormap_dict(quantile_df, measure)
    colormap = create_scale(metadata_full,colormap_dict,measure)
    display_scale = create_matplot_scale(metadata_full,colormap_dict,measure)
    geojson = assign_json(year,area_type,state)
    
    map = folium.Map(location=[30, 0], zoom_start=2)
    folium.GeoJson(
        geojson,
        name='geojson',
        style_function=lambda feature: style_function(feature,colormap,measure),
        highlight_function=lambda x: {'weight':3, 'fillOpacity':0.9},  # Highlight effect when mouse hovers
        tooltip=folium.features.GeoJsonTooltip(fields=['GEO_NAME', measure], aliases=['Name', 'Value'], labels=True, sticky=True),
        popup = folium.features.GeoJsonPopup(fields=['GEO_NAME', measure], aliases=['Name', 'Value'], labels=True, parse_html=True),
    ).add_to(map)

    svg_style = '<style>svg#legend {background-color: GhostWhite;border-style: solid;border-width: 1px;}</style>'
    #map.get_root().header.add_child(folium.Element(svg_style))
    #colormap.add_to(map)
    
    Geocoder().add_to(map)

    html_string = map.get_root().render()
    
    return html_string, display_scale

#### Function to create Notes

In [10]:
print("The variables in the metadata are:", metadata_full.columns)
domain_options=metadata['Domain1'].unique().tolist()
print(domain_options)
notes_options=metadata['Variable_Label'].tolist()
print(notes_options)

The variables in the metadata are: Index(['Variable Name', 'YLABEL', 'Card', 'Group', 'Decimal', 'Per_Dollars',
       'Domain1', 'Domain2', 'Variable_Label', 'Variable_Description',
       'Interpretation', 'Source', 'Data_Description', 'Data_Provenance',
       'Data_link', 'Metadata_link', 'Coverage', 'Frequency', 'Unit',
       'Geography', 'Uncertainty', 'Citation'],
      dtype='object')
['General', 'Health & Nutrition', 'Infrastructure, Housing, & Transportation', 'Income, Poverty, & Social Services', 'Education', 'Public Safety', 'Disaster Prevention and Relief', 'Environmnet & Natural Resources', 'Economic Vitality']
['Total Population', 'Median Age', 'Median Age Male', 'Median Age Female', 'Percent under 5', 'Percent 5 to 9', 'Percent 10 to 14', 'Percent 15 to 19', 'Percent 20 to 24', 'Percent 25 to 29', 'Percent 30 to 34', 'Percent 35 to 39', 'Percent 40 to 44', 'Percent 45 to 49', 'Percent 50 to 54', 'Percent 55 to 59', 'Percent 60 to 64', 'Percent 65 to 69', 'Percent 70 to

#### Functions to create the time series

In [11]:
def create_empty_fig():
    fig = go.Figure()
    fig.update_layout(title="No Data Available", showlegend=False)
    return fig

In [12]:
def create_time_series(color_scheme,session_data,domain,title):
    try: 
        filtered_vars_dict = get_domain_list(domain,title)
        
        #filtered_vars_dict['YEAR'] = 'Year'
        #filtered_vars_dict['GEO_NAME'] = 'Geographic Area'
        
        filtered_vars = list(filtered_vars_dict.keys())
        filtered_vars_label = list(filtered_vars_dict.values())
        df_session_data = pd.DataFrame(session_data)
        df_session_data = df_session_data.dropna(subset=filtered_vars)
        
        if df_session_data[filtered_vars].isnull().all().all():
            return create_empty_fig(), 
        
        ylabel=metadata_full.loc[metadata_full['Variable Name'] == str(filtered_vars[0]), 'YLABEL'].iloc[0]
        geo_name=df_session_data['GEO_NAME'].iloc[0]

        #f"Warning: No data available for the selections. Data might be available for other geographies or years."
        
        # Sort by year 
        df_session_data = df_session_data.sort_values(["YEAR"], ascending=[True])
        
         # Get the colors for the selected color scheme
        colors = color_schemes.get(color_scheme, px.colors.qualitative.Plotly)
        
        if len(filtered_vars)==1:
            fig = px.line(df_session_data, x='YEAR', y=filtered_vars, markers=True,
                          labels={'YEAR': 'Year'},
                          color_discrete_sequence=colors)
            fig.for_each_trace(lambda t: #print(t.hovertemplate)
                               t.update(name = filtered_vars_dict[t.name],
                                      legendgroup = filtered_vars_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, filtered_vars_dict[t.name])
                                     )
                              )
            fig.update_layout(#title=f'{title} for {geo_name}',
                             legend_title=f'Variables',
                              yaxis_title=ylabel)
            #fig.show()
            return fig,

        else:
            fig = px.line(df_session_data, x='YEAR', y=filtered_vars,
                          labels={'YEAR': 'Year'},
                          color_discrete_sequence=colors)
            #fig.update_xaxes(tickmode='array', tickvals=sorted(selected_years))
            #fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
            fig.for_each_trace(lambda t: t.update(name = filtered_vars_dict[t.name],
                                      legendgroup = filtered_vars_dict[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, filtered_vars_dict[t.name])
                                     )
                              )
            fig.update_layout(#title=f'{title} for {geo_name}',
                             legend_title=f'Variables',
                              yaxis_title=ylabel)
            #fig.show()
            return fig,
                                                                                                    
    except Exception as e:
        return create_empty_fig(),
        #, f"An error occurred while generating the plot: {str(e)}"

#Orginal code for reference
"""
        filtered_vars = varcats[varcats['CATEGORY'] == selected_category]['VARIABLE']
        filtered_data = merged_data[merged_data['GEO_ID'].isin(selected_geo_ids) & merged_data['YEAR'].isin(selected_years)][['YEAR', 'GEO_ID', 'GEO_NAME'] + list(filtered_vars)]
        filtered_data = filtered_data.dropna(subset=filtered_vars)
        if filtered_data[filtered_vars].isnull().all().all():
            return {}, f"Warning: No data available for the selections. Data might be available for other geographies or years."
        
        melted_data = filtered_data.melt(id_vars=['YEAR', 'GEO_ID', 'GEO_NAME'], var_name='Variable', value_name='Value')
        
        # Sort by year and category in varcats
        variable_order = varcats[varcats['CATEGORY'] == selected_category]['VARIABLE'].tolist()
        melted_data['Variable'] = pd.Categorical(melted_data['Variable'], categories=variable_order, ordered=True)
        melted_data = melted_data.sort_values(["YEAR", "Variable"], ascending=[True, True])

        melted_data['Variable'] = melted_data['Variable'].map(column_label_map)  # Map variable names to labels from varcats
        ylabel = ylabel_map.get(selected_category, 'Value')  # Get Y-axis label from varcats

        # Check if one of the ratings variables
        for col in variable_order:
            if col in ratings_cols:
                melted_data['Value'] = pd.Categorical(melted_data['Value'], categories=ratings_vars, ordered=True)
                melted_data = melted_data.sort_values(['Value', 'YEAR', 'GEO_ID'])

        # Drop NA values
        melted_data = melted_data.dropna(subset=['Value'])
    
        # Check if sum of values of the variables in the selected category equals 1 for each bar with a tolerance of 0.3 (i.e., 99.7 and 100.3 are acceptable)
        numeric_vars = melted_data[melted_data['Value'].apply(lambda x: isinstance(x, (int, float)))]
        barmode = 'group'
        if not numeric_vars.empty:
            grouped_data = numeric_vars.groupby(['GEO_NAME', 'YEAR', 'Variable']).sum().reset_index()
            sum_check = grouped_data.groupby(['GEO_NAME', 'YEAR'])['Value'].sum().reset_index()
            sum_check['Value'].replace(0, 100, inplace=True)
            barmode = 'stack' if sum_check['Value'].apply(lambda x: np.isclose(x, 100, atol=.3)).all() else 'group'

        # Get the colors for the selected color scheme
        colors = color_schemes.get(color_scheme, px.colors.qualitative.Plotly)

        if plot_type == 'time_series':
            if len(filtered_vars)==1:
                fig = px.line(melted_data, x='YEAR', y='Value', color='GEO_NAME', markers=True, labels={'Value': ylabel, 'YEAR': 'Year'}, color_discrete_sequence=colors)
            else:
                fig = px.line(melted_data, x='YEAR', y='Value', facet_col='GEO_NAME', color='Variable', markers=True, labels={'Value': ylabel, 'YEAR': 'Year'}, color_discrete_sequence=colors)
            fig.update_xaxes(tickmode='array', tickvals=sorted(selected_years))
            fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        else:
            if len(filtered_vars)==1:
                fig = px.bar(melted_data, x='GEO_NAME', y='Value', color='GEO_NAME', barmode=barmode, facet_col='YEAR', labels={'Value': ylabel, 'YEAR': 'Year', 'GEO_NAME': 'Geography'}, color_discrete_sequence=colors)
            else:
                fig = px.bar(melted_data, x='GEO_NAME', y='Value', color='Variable', barmode=barmode, facet_col='YEAR', labels={'Value': ylabel, 'YEAR': 'Year', 'GEO_NAME': 'Geography'}, color_discrete_sequence=colors)
                fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
            

        fig.update_layout(title=f'{selected_category} for Selected Geographies and Years')
        return fig, ""
    except Exception as e:
        return {}, f"An error occurred while generating the plot: {str(e)}"
    """ 


'\n        filtered_vars = varcats[varcats[\'CATEGORY\'] == selected_category][\'VARIABLE\']\n        filtered_data = merged_data[merged_data[\'GEO_ID\'].isin(selected_geo_ids) & merged_data[\'YEAR\'].isin(selected_years)][[\'YEAR\', \'GEO_ID\', \'GEO_NAME\'] + list(filtered_vars)]\n        filtered_data = filtered_data.dropna(subset=filtered_vars)\n        if filtered_data[filtered_vars].isnull().all().all():\n            return {}, f"Warning: No data available for the selections. Data might be available for other geographies or years."\n        \n        melted_data = filtered_data.melt(id_vars=[\'YEAR\', \'GEO_ID\', \'GEO_NAME\'], var_name=\'Variable\', value_name=\'Value\')\n        \n        # Sort by year and category in varcats\n        variable_order = varcats[varcats[\'CATEGORY\'] == selected_category][\'VARIABLE\'].tolist()\n        melted_data[\'Variable\'] = pd.Categorical(melted_data[\'Variable\'], categories=variable_order, ordered=True)\n        melted_data = melted_data

#### Function to create the bar Chart

In [13]:
def create_bar_chart(color_scheme,session_data,domain,title):
    try:
        #ideally this should be written to check to see values add up to 100, I ran out of time and hard coded general tab for demo
        if title in ['Age','Race and Hispanic origin']:
            graphtype = 'stack'
        else:
            graphtype = 'group'
        filtered_vars_dict = get_domain_list(domain,title)
        #filtered_vars_dict['YEAR'] = 'Year'
        #filtered_vars_dict['GEO_NAME'] = 'Geographic Area'
        
        filtered_vars = list(filtered_vars_dict.keys())
        filtered_vars_label = list(filtered_vars_dict.values())
        
        df_session_data = pd.DataFrame(session_data)
        df_session_data = df_session_data.dropna(subset=filtered_vars)

        df_session_data.rename(columns=filtered_vars_dict, inplace=True)


        #df_session_data = df_session_data.dropna(subset=filtered_vars)
        if df_session_data[filtered_vars_label].isnull().all().all():
            return create_empty_fig(),
        #f"Warning: No data available for the selections. Data might be available for other geographies or years."
  
        ylabel=metadata_full.loc[metadata_full['Variable_Label'] == str(filtered_vars_label[0]), 'YLABEL'].iloc[0]
        
        geo_name=df_session_data['GEO_NAME'].iloc[0]
        # Sort by year 
        df_session_data = df_session_data.sort_values(["YEAR"], ascending=[True])
        
         # Get the colors for the selected color scheme
        colors = color_schemes.get(color_scheme, px.colors.qualitative.Plotly)
        
        if len(filtered_vars)==1:
            fig = px.bar(df_session_data,x='YEAR',y=filtered_vars_label[0],
                         labels={'value':'Value', 'YEAR': 'Year'},
                         color_discrete_sequence=colors)
            #fig.show()
            #fig.update_layout(title=f'{title} for {geo_name}')
            return fig,
            
        else:
            fig = px.bar(df_session_data,x='YEAR',
                         y=filtered_vars_label,
                         labels={'value':'Value', 'YEAR': 'Year','variable':'variable'},
                         color_discrete_sequence=colors,
                         barmode = graphtype
                        )
            fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
            fig.update_layout(#title=f'{title} for {geo_name}',
                             legend_title=f'Variables',
                              yaxis_title=ylabel)
            #fig.show()
            return fig,
                                                                                                    
    except Exception as e:
        return create_empty_fig(),
    #f"An error occurred while generating the plot: {str(e)}"

#### Function to create tables

In [14]:
def create_table(domain,group,session_data):
    df_session_data = pd.DataFrame(session_data)
    try:
        filtered_vars=metadata['Variable Name'][(metadata['Group']==group)&(metadata['Domain1']==domain)]
        filtered_labels=metadata['Variable_Label'][metadata['Group']==group]
        
        df_session_data = df_session_data[['GEO_NAME', 'GEO_ID', 'YEAR'] + list(filtered_vars)]
        df_session_data = df_session_data.dropna(subset=list(filtered_vars))
                
        if df_session_data.empty:
            return html.Div([
                html.P("Warning: No data are available for your current selections.")
            ],style={'color': 'red', 'fontWeight': 'bold'}
            )
        
        # Map variable names to labels
        label_map = dict(zip(metadata['Variable Name'], metadata['Variable_Label']))
        df_session_data.rename(columns=label_map, inplace=True)
        df_session_data.rename(columns={'GEO_NAME':'Geography','GEO_ID':'Geographic ID','YEAR':'Year'}, inplace=True)
        # Formats values based on the metadata
        for col in df_session_data.columns:
            if col in ['GEO_NAME', 'GEO_ID', 'YEAR']:
                pass
            else:
                try:
                    decimal = metadata_full.loc[metadata_full['Variable_Label'] == col, 'Decimal'].values[0]
                    #commenting out but add back and add to format value function 
                    #percent_dollar = metadata_full.loc[metadata_full['Variable_Label'] == col, 'Per_Dollars'].values[0]
                    df_session_data[col] = df_session_data[col].apply(lambda x: format_value(x,decimal))
                except:
                    pass
                    
        return [dash_table.DataTable
                (columns=[{'name': col, 'id': col} for col in df_session_data.columns],
                 data=df_session_data.to_dict('records'),
                 sort_action='native', #Enables column sorting
                 style_table={'textTransform': 'none','overflowX': 'auto', 'maxWidth': '95%'},  # Adjust the maxWidth as needed
                 style_cell={
                     'textTransform': 'none',
                     'whiteSpace': 'normal',
                     'height': 'auto',
                     'textAlign': 'center',  # Center align the content
                     'fontSize': 14
                 },
                 style_header={
                     'backgroundColor': 'rgb(230, 230, 230)',
                     'fontWeight': 'bold',
                     'textAlign': 'center'  # Center align the header
                 },
                 export_format='csv',  #To downlaod data table as a csv file
                 export_headers='display',  #Keep formatting for csv file as it displays on screen
                 # Format number columns
                 style_data={
                     'if': {'column_type': 'numeric'},
                     'textAlign': 'right',
                 }
                )
               ]
    except Exception as e:
        print(f"Error in create_table: {e}")
        return html.Div(
            html.P(f"An error occurred while generating the table: {str(e)}"),
            style={'color': 'red', 'fontWeight': 'bold'}
        )

#### Functions for webpage to Compare with other communities

In [15]:
merged_data_file_path = os.path.join(base_dir, "Data", "Final Data", "Merged", "merged_data.csv")
varcats_file_path = os.path.join(base_dir, "Data", "varcats_v5.csv")

# Load datasets.
merged_data = pd.read_csv(merged_data_file_path, low_memory=False)
varcats = pd.read_csv(varcats_file_path, encoding='cp1252')  # Use encoding to decode special symbols (e.g., the degree symbol in TEMPERATURE).

# Combine GEO_NAME and GEO_ID for display in a dropdown.
geo_name_id = merged_data[['GEO_ID', 'GEO_NAME']].drop_duplicates()
geo_name_id['combined'] = geo_name_id.apply(lambda x: f"{x['GEO_NAME']} ({x['GEO_ID']})", axis=1)

# Define the ordered categories and the columns these are used on.
ratings_vars = [
    'Data Unavailable', 'No Rating', 'Insufficient Data', 'Very Low', 
    'Relatively Low', 'Relatively Moderate', 'Relatively High', 'Very High'
]
ratings_cols = ["RISK_RATNG", "RESL_RATNG"]

# Convert the specified columns into ordered categorical columns.
for col in ratings_cols:
    merged_data[col] = pd.Categorical(merged_data[col], categories=ratings_vars, ordered=True)

# Preserve original values for 'BALANCE' and temperature columns.
balance_temp = merged_data[["BALANCE", "TMIN_F", "TMAX_F"]]

# Function to replace negative numbers with NaN.
def replace_negatives_with_nan(value):
    if isinstance(value, (int, float)) and value < 0:
        return np.nan
    else:
        return value

# Apply the function to each element in the DataFrame.
merged_data = merged_data.applymap(replace_negatives_with_nan)

# Restore the original 'BALANCE' and temperature values to retain negatives.
merged_data[["BALANCE", "TMIN_F", "TMAX_F"]] = balance_temp

# Define color schemes for different types of color blindness and monochrome.
color_schemes = {
    'default': px.colors.qualitative.Plotly,
    'monochrome': ['#000000', '#4d4d4d', '#999999', '#b3b3b3', '#cccccc'],  # Shades of gray.
    'deuteranopia': ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00'],  # Difficulty with green.
    'protanopia': ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999'],  # Difficulty with red.
    'tritanopia': ['#0000ff', '#ffff00', '#008000', '#ff0000', '#7a288a', '#4b0082', '#2f4f4f', '#008080', '#800080']   # Blue-yellow color blindness.
}

# Extract unique categories from varcats, excluding 'INDEPENDENT'.
categories = varcats[varcats['CATEGORY'] != 'INDEPENDENT']['CATEGORY'].unique()
geo_ids = merged_data['GEO_ID'].unique()
years = merged_data['YEAR'].unique()

# Create dictionaries to map variable names to labels and Y-axis labels.
column_label_map = dict(zip(varcats['VARIABLE'], varcats['LABEL']))
ylabel_map = dict(zip(varcats['CATEGORY'], varcats['YLABEL']))

C:\Users\Colin\AppData\Local\Temp\ipykernel_19836\3656838762.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_data = merged_data.applymap(replace_negatives_with_nan)


#### For cards at the top of each domain webpage

##### Getting content for individual cards

In [16]:
#Get a count of cards by domain
for domain in metadata['Domain1'].unique():
    print(metadata_cards['Domain1'][(metadata_cards['Domain1']==domain)].value_counts())

Domain1
General    18
Name: count, dtype: int64
Domain1
Health & Nutrition    10
Name: count, dtype: int64
Domain1
Infrastructure, Housing, & Transportation    10
Name: count, dtype: int64
Domain1
Income, Poverty, & Social Services    5
Name: count, dtype: int64
Domain1
Education    7
Name: count, dtype: int64
Domain1
Public Safety    7
Name: count, dtype: int64
Domain1
Disaster Prevention and Relief    14
Name: count, dtype: int64
Domain1
Environmnet & Natural Resources    8
Name: count, dtype: int64
Domain1
Economic Vitality    7
Name: count, dtype: int64


##### Functions used in the cards

In [17]:
#see if the file path for icon exists
def check_file_existence(file_path):
    return os.path.isfile(file_path)

#returns a png filepath based on the the title of the card
def get_icon(title):
    generic_path="/data/discover/Code/assets/generic.png"
    path = "/data/discover/Code/assets/" + title + ".png"
    if check_file_existence(path):
        pil_image = Image.open(path)
        return pil_image
    else:
        pil_image = Image.open(generic_path)
        return pil_image
    
#Gets value for card from session_data, which is determined based on user selection in find my community tools.
def return_card_value(session_data,measure):
    df = pd.DataFrame(session_data)
    #These return data not available if nothing has been selected or if no rows contain values
    if df.empty:
        return "Select your community"
    elif df[measure].isnull().all():
        return("Estimate not available")
    else:
        df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')
        df = df.dropna(subset=['YEAR'])
        # Filter out rows with null name values, find the row with the maximum year, and get the corresponding name
        max_age_row = df[df[measure].notnull()]['YEAR'].idxmax()
        name_for_max_age = df.loc[max_age_row,measure]
        return name_for_max_age

In [18]:
#Function to create cards
def create_card(title,measure,interpretation,source,anchor,navcolor,session_data):
    anchorlink="#"+anchor
    #Gets title, icon, and card values
    title_str =title.iloc[0]
    pil_Image = get_icon(title_str)
    value = return_card_value(session_data,measure)
    #get formatting inputs from metadata
    decimal = (metadata_full.loc[metadata_full['Variable Name'] == measure, 'Decimal'].values[0])
    percent_dollar = (metadata_full.loc[metadata_full['Variable Name'] == measure, 'Per_Dollars'].values[0])
    
    #format the values using function
    value = format_value(value,decimal,percent_dollar)
    
    #returns card in dash format
    return [
        dbc.CardImg(src=pil_Image,
                    top=True,
                    style={"height":"100px","width":"100px"}),
        dbc.CardBody(
            [
                ####Card Header
                html.H5(title, className="card-title",style={"color": navcolor}),
                ####Card Estimate
                ####Currently uses the testing df - small rather than the full df - df
                html.B(html.I(value,style={"font-size": "25px", "color": navcolor})),
                ####Interpretation of Estimate
                html.P(interpretation,style={"color": navcolor}),
                ####Link to go to another webpage with visualizations for the estimate
                dbc.Nav([dbc.NavLink("See detailed graphs", href=anchorlink, active="exact",\
                                     style={"color": navcolor, "text-decoration": "underline"},\
                                     external_link=True),]),
                html.I(["Source: ", html.I(source)],style={"color": navcolor, "font-size": "13px"})
        ]
    ),
]

##### Building rows of 4 cards
Note: The maximum number of cards is 18 so there are 5 of the below functions.

In [19]:
#Functions to create cards for all domains
##Row 1
def row1(domain,session_data):
    return [
        dbc.Col(dbc.Card(create_card(session_data=session_data,
                                     title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[0]],
                                     measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[0],
                                     interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[0]],
                                     source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[0]],
                                     anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[0]],
                                     navcolor="white"),
                         color="#2166ac"),width=3),
        dbc.Col(dbc.Card(create_card(session_data=session_data,
                                     title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[1]],
                                     measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[1],
                                     interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[1]],
                                     source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[1]],
                                     anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[1]],
                                     navcolor="black"),
                         color="#67a9cf"),width=3),
        dbc.Col(dbc.Card(create_card(session_data=session_data,
                                     title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[2]],
                                     measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[2],
                                     interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[2]],
                                     source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[2]],
                                     anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[2]],
                                     navcolor="black"),
                         color="#d1e5f0"),width=3),
        dbc.Col(dbc.Card(create_card(session_data=session_data,
                                     title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[3]],
                                     measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[3],
                                     interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[3]],
                                     source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[3]],
                                     anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[3]],
                                     navcolor="black"),
                         color="#fddbc7"),width=3),
    ]

##Row 2
def row2(domain,session_data):
    condition=metadata_cards['Domain1'][(metadata_cards['Domain1']==domain)].value_counts().item()
    if (condition>=8):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4],
                                         interpretation=metadata_cards['Variable_Label'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         navcolor="black"),
                             color="#ef8a62"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         navcolor="white"),
                             color="#b2182b"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         navcolor="white"),
                             color="#2166ac"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[7]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[7],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[7]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[7]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[7]],
                                         navcolor="black"),
                             color="#67a9cf"),width=3),
        ]
    elif (condition==7):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4],
                                         interpretation=metadata_cards['Variable_Label'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         navcolor="black"),
                             color="#ef8a62"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         navcolor="white"),
                             color="#b2182b"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[6]],
                                         navcolor="white"),
                             color="#2166ac"),width=3),
        ]
    elif (condition==6):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4],
                                         interpretation=metadata_cards['Variable_Label'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         navcolor="black"),
                             color="#ef8a62"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[5]],
                                         navcolor="white"),
                             color="#b2182b"),width=3),
        ]
    elif (condition==5):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4],
                                         interpretation=metadata_cards['Variable_Label'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[4]],
                                         navcolor="black"),
                             color="#ef8a62"),width=3),
        ]
    
##Row 3
def row3(domain,session_data):
    condition=metadata_cards['Domain1'][(metadata_cards['Domain1']==domain)].value_counts().item()
    if (condition>=12):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         navcolor="black"),
                             color="#d1e5f0"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         navcolor="black"),
                             color="#fddbc7"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         navcolor="black"),
                             color="#ef8a62"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[11]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[11],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[11]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[11]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[11]],
                                         navcolor="white"),
                             color="#b2182b"),width=3),
    ]
    elif (condition==11):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         navcolor="black"),
                             color="#d1e5f0"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         navcolor="black"),
                             color="#fddbc7"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[10]],
                                         navcolor="black"),
                             color="#ef8a62"),width=3),
    ]
    elif (condition==10):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         navcolor="black"),
                             color="#d1e5f0"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[9]],
                                         navcolor="black"),
                             color="#fddbc7"),width=3),
    ]
    elif (condition==9):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[8]],
                                         navcolor="black"),
                             color="#d1e5f0"),width=3),
        ]
    
##Row 4
def row4(domain,session_data):
    condition=metadata_cards['Domain1'][(metadata_cards['Domain1']==domain)].value_counts().item()
    if (condition>=16):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         navcolor="white"),
                             color="#2166ac"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         navcolor="black"),
                             color="#67a9cf"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         navcolor="black"),
                             color="#d1e5f0"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[15]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[15],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[15]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[15]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[15]],
                                         navcolor="black"),
                             color="#fddbc7"),width=3),
        ]
    elif (condition==15):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         navcolor="white"),
                             color="#2166ac"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         navcolor="black"),
                             color="#67a9cf"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[14]],
                                         navcolor="black"),
                             color="#d1e5f0"),width=3),
            ]
    elif (condition==14):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         navcolor="white"),
                             color="#2166ac"),width=3),
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[13]],
                                         navcolor="black"),
                             color="#67a9cf"),width=3),
            ]
    elif (condition==13):
        return [
            dbc.Col(dbc.Card(create_card(session_data=session_data,
                                         title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12],
                                         interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[12]],
                                         navcolor="white"),
                             color="#2166ac"),width=3),
            ]
    
##Row 5
def row5(domain,session_data):
    return [
        dbc.Col(dbc.Card(create_card(session_data=session_data,
                                     title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[16]],
                                     measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[16],
                                     interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[16]],
                                     source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[16]],
                                     anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[16]],
                                     navcolor="black"),
                         color="#ef8a62"),width=3),
        dbc.Col(dbc.Card(create_card(session_data=session_data,
                                     title=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[17]],
                                     measure=metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[17],
                                     interpretation=metadata_cards['Interpretation'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[17]],
                                     source=metadata_cards['Source'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[17]],
                                     anchor=metadata_cards['Group'][metadata_cards['Variable Name']==metadata_cards['Variable Name'][metadata_cards['Domain1']==domain].iloc[17]],
                                     navcolor="white"),
                         color="#b2182b"),width=3),
    ]

##### Putting all rows together

In [20]:
def compile_cards(domain,session_data):
    condition=metadata_cards['Domain1'][(metadata_cards['Domain1']==domain)].value_counts().item()
    if (condition>16):
        return html.Div(
            [
                dbc.Row(row1(domain,session_data),className="mb-4",),
                dbc.Row(row2(domain,session_data),className="mb-4",),
                dbc.Row(row3(domain,session_data),className="mb-4",),
                dbc.Row(row4(domain,session_data),className="mb-4",),
                dbc.Row(row5(domain,session_data),className="mb-4",),
            ]
        )
    elif (condition>12):
        return html.Div(
            [
                dbc.Row(row1(domain,session_data),className="mb-4",),
                dbc.Row(row2(domain,session_data),className="mb-4",),
                dbc.Row(row3(domain,session_data),className="mb-4",),
                dbc.Row(row4(domain,session_data),className="mb-4",),
            ]
        )
    elif (condition>8):
        return html.Div(
            [
                dbc.Row(row1(domain,session_data),className="mb-4",),
                dbc.Row(row2(domain,session_data),className="mb-4",),
                dbc.Row(row3(domain,session_data),className="mb-4",),
            ]
        )
    else:
        return html.Div(
            [
                dbc.Row(row1(domain,session_data),className="mb-4",),
                dbc.Row(row2(domain,session_data),className="mb-4",),
            ]
        )


#### For accordions at the bottom of each domain webpage

##### Getting content for accordion items

In [21]:
#Function to create accordions
def create_accordionitem(title,domain,session_data):
    measures=metadata['Variable Name'][metadata['Group']==title]
    
    #creates unique ids for callbacks
    id=title
    item_id="item-"+title
    map_year_input_id = f"{title}-map-year-input"
    map_area_type_input_id = f"{title}-map-area-type-input"
    map_measure_input_id = f"{title}-map-measure-input"
    map_state_input_id = f"{title}-map-state-input"
    update_map_button_id = f"{title}-update-map-button"
    map_output_id = f"{title}-map"
    scale_output_id = f"{title}-scale"
    
    #Settings for charts
    domain_input_id = f"{title}-domain-input"
    title_input_id = f"{title}-title-input"
    color_scheme_input_id = f"{title}-color_scheme-input"
    times_series_output_id = f"{title}-time-series" 
    update_time_series_id = f"{title}-update-time-series-button"
    
    bar_chart_output_id = f"{title}-bar-chart" 
    update_bar_chart_id = f"{title}-bar-chart-series-button"
    bar_chart_color_scheme_input_id = f"{title}-bar-chart-color_scheme-input"
    
    return dbc.AccordionItem(
        [
            html.H5('Map',style={"color": "#CF5008"}),
            html.I('Instructions:'),
            html.Br(),
            html.I("1. Select the year, type of area, and measure you would like to map. To see a map at \
            the Census tract level, you must also select your state."),
            html.Br(),
            html.I("2. Click the Update Map button."),
            dbc.Row([html.P("")]),
            dbc.Row([
                #These are the dropdown slections, inputs determined above, get domain uses a function to determine
                dbc.Col([html.I("Select Year:"),dcc.Dropdown(options=years_list,
                                             value=years_list[0],
                                             id=map_year_input_id#Will this create issue with other accordians?
                                             #Could make this part more dynamic so it based on data rather than hard coding
                                            )
                        ]
                       ),
                dbc.Col([html.I("Select Area Type:"),dcc.Dropdown(options=area_type_list,
                                                  value='state',
                                                  id=map_area_type_input_id)]
                       ),
                dbc.Col([html.I("Select Measure:"),dcc.Dropdown(options=get_domain_list(domain,title),
                                                                value=metadata_cards['Variable Name'][metadata_cards['Group']==title].iloc[0],
                                                                id=map_measure_input_id)]
                       ),
                dbc.Col([html.I("Select State (required only if Area Type is Tract):"),dcc.Dropdown(options=fips_state_list,
                                                                                    id=map_state_input_id)
                    ]),
                dbc.Col(html.Button("Update Map", id=update_map_button_id))
            ]),
            #Below is the map
            html.Iframe(id=map_output_id, srcDoc=None, width='100%', height='500px'),
            html.Div(id=scale_output_id),
            html.Br(),
            html.Br(),
            html.H5('Time Series',style={"color": "#CF5008"}),
            html.I('Instructions:'),
            html.Br(),
            html.I("1. Select the color scheme you would like to use."),
            html.Br(),
            html.I("2. Click the Update Time Series button."),
            dbc.Row([html.P("")]),
            html.Div([
                # Color scheme selection for color blindness
                html.Label("Select Color Scheme:"),
                dcc.Dropdown(id=color_scheme_input_id,
                             options=[
                                 {'label': 'Default', 'value': 'default'},
                                 {'label': 'Monochrome (Black & White)', 'value': 'monochrome'},
                                 {'label': 'Deuteranopia (Issues with Green)', 'value': 'deuteranopia'},
                                 {'label': 'Protanopia (Issues with Red)', 'value': 'protanopia'},
                                 {'label': 'Tritanopia (Issues with Blue)', 'value': 'tritanopia'}
                             ],
                             value='default'
                            ),
                html.Button("Update Time Series", id=update_time_series_id)
            ]),
            dcc.Input(id=title_input_id,value=title,type='text',style={'display':'none'}),
            dcc.Input(id=domain_input_id,value=domain,type='text',style={'display':'none'}),
            dcc.Graph(id=times_series_output_id),
            html.H5('Bar Chart',style={"color": "#CF5008"}),
            html.I('Instructions:'),
            html.Br(),
            html.I("1. Select the color scheme you would like to use."),
            html.Br(),
            html.I("2. Click the Update Bar Chart button."),
            dbc.Row([html.P("")]),
            html.Div([
                # Color scheme selection for color blindness
                html.Label("Select Color Scheme for Color Blindness:"),
                dcc.Dropdown(id=bar_chart_color_scheme_input_id,
                             options=[
                                 {'label': 'Default', 'value': 'default'},
                                 {'label': 'Monochrome (Black & White)', 'value': 'monochrome'},
                                 {'label': 'Deuteranopia (Issues with Green)', 'value': 'deuteranopia'},
                                 {'label': 'Protanopia (Issues with Red)', 'value': 'protanopia'},
                                 {'label': 'Tritanopia (Issues with Blue)', 'value': 'tritanopia'}
                             ],
                             value='default'
                            ),
                html.Button("Update Bar Chart", id=update_bar_chart_id)
            ]),
            dcc.Graph(id=bar_chart_output_id),
            # Warning message
            #html.Div(id='warning-message', style={'color': 'red', 'fontWeight': 'bold'}),
            html.H5('Table',style={"color": "#CF5008"}),
            html.Div(create_table(domain,title,session_data)),
        ],
        title=title,
        id=id,
        item_id=item_id,
    )

##### Putting accordions together

In [22]:
def compile_accordion(domain,session_data):
    list=[f"item-{num}" for num in metadata_accordions['Group'][metadata_accordions['Domain1']==domain]]
    return html.Div(
        dbc.Accordion(
            [create_accordionitem(title,domain,session_data) for title in metadata_accordions['Group'][metadata_accordions['Domain1']==domain]],
            id="accordion",
            active_item=list,
            #start_collapsed=True,
        )
    )

### Application

In [ ]:
#Initialize the app
app=Dash(__name__,
         ##Allows for code in a pages folder to be called by this program
         ###Currently not used and commented out
         #use_pages=True,
         ##For other stylesheets to use for the external_stylesheets option, see this link:
         ##https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/explorer/
         external_stylesheets=[dbc.themes.LUX])

#Header
heading = html.H1("Census Data Explorer for Grant Writers")

#Sidebar with links to each domain and a Contact Us link
##Style arguments
SIDEBAR_STYLE = {
    ###Using position:fixed
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    ###Using a fixed width
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f7f7f7",
}

##Layout
sidebar = html.Div(
    [
        html.B(html.I("Choose any topic",style={"color": "#A43E05"})),
        dbc.Nav(
            [
                ###Lists all domains in order of appearance with their link suffixes specified
                dbc.NavLink("General", href="/", active="exact"),
                dbc.NavLink("Housing, Infrastructure, & Transportation", href="/housing_infrastructure_transportation",\
                            active="exact"),
                dbc.NavLink("Health & Nutrition", href="/health_nutrition", active="exact"),
                dbc.NavLink("Education", href="/education", active="exact"),
                dbc.NavLink("Income, Poverty & Social Services", href="/income_poverty_socialservices", active="exact"),
                dbc.NavLink("Public Safety", href="/publicsafety", active="exact"),
                dbc.NavLink("Economic Vitality", href="/economicvitality", active="exact"),
                dbc.NavLink("Environment & Natural Resources", href="/environment_naturalresources", active="exact"),
                dbc.NavLink("Disaster Prevention & Relief", href="/disasterprevention_relief", active="exact")
            ],
            ###Lists the links in the sidebar vertically
            vertical=True,
            ###Makes the links in the sidebar appear as "pills" or buttons.
            ####Not currently used and commented out
            #pills=True,
        ),
        ###Inserts a break between domain links and Shannan's code page
        html.Br(),
        dbc.NavLink(html.B("Customize your detailed graphs by geography and year"),
                    href="/compare", active="exact",style={"color": "blue", "text-decoration": "underline"}),
        ###Inserts a break between Shannan's code page and the Notes link
        html.Br(),
        dbc.NavLink(html.B("Notes"), href="/notes", active="exact",style={"color": "blue", "text-decoration": "underline"}),
        ###Inserts a break between the Notes link and the Contact Us link
        html.Br(),
        ###Creates a Contact Us link with a Suggestions/Questions? header
        ####Both are smaller in font size and underlined 
        html.P(["Suggestions/Questions? ",
               html.A("Click here to contact us.", href="https://ask.census.gov/support/case", target="_blank",style={"color": "blue", "text-decoration": "underline"}),
               ],style={"font-size": "13px"}),
        ####Cites where icons were obtained
        html.P(["Icons designed by ",
                html.A("freepik", href="https://www.freepik.com/", target="_blank",style={"color": "blue", "text-decoration": "underline"}),
               ],style={"font-size": "13px"}),
 
    ],
    ###Include style arguments from above
    style=SIDEBAR_STYLE,
)

##Add controls to build the interaction with visualization
###Callbacks for the accordion
@app.callback(
    Output("accordion-contents", "children"),
    [Input("accordion", "active_item")],
)
def change_item(item):
    return f"Item selected: {item}"

###Callbacks for the map
####year, area_type, measure, state=False
def update_map_callback(title):
    @app.callback([Output(f"{title}-map",'srcDoc'),
                   Output(f"{title}-scale",'children')],
                  [Input(f"{title}-update-map-button", 'n_clicks')],
                  [State(f"{title}-map-year-input",'value'),
                   State(f"{title}-map-area-type-input",'value'),
                   State(f"{title}-map-measure-input",'value'),
                   State(f"{title}-map-state-input",'value')]
                 )
    def update_map(n_clicks, year, area_type, measure, state=False):
        if n_clicks is None:
            return dash.no_update
        else:
            return create_map(year, area_type, measure, state)

###Create_map(year, area_type, measure, state)
for title in title_list:
    update_map_callback(title)
    

#creates needed callbacks for each time series in accordian
def update_time_series_callback(title):
    @app.callback([Output(f"{title}-time-series" , 'figure')],
                  #Output('warning-message', 'children')],
                  [Input('session_data', 'data'),
                  Input(f"{title}-update-time-series-button",'n_clicks')],
                  [State(f"{title}-color_scheme-input", 'value'),
                  State(f"{title}-domain-input",'value'),
                  State(f"{title}-title-input",'value')]
                 )
    def update_time_series(session_data,n_clicks,color_scheme,domain,title):
        return create_time_series(color_scheme,session_data,domain,title)
    """
        if n_clicks is None:
            return create_time_series(color_scheme,session_data,domain,title)
            #return dash.no_update
        else:
            #print("updating")
            #print(color_scheme,domain,title)
            return create_time_series(color_scheme,session_data,domain,title)
    """
for title in title_list:
    update_time_series_callback(title)

#creates needed callbacks for each bar chart in accordian
def update_bar_chart_callback(title):
    @app.callback([Output(f"{title}-bar-chart" , 'figure')],
                  #Output('warning-message', 'children')],
                  [Input('session_data', 'data'),
                  Input(f"{title}-bar-chart-series-button",'n_clicks')],
                  [State(f"{title}-bar-chart-color_scheme-input", 'value'),
                  State(f"{title}-domain-input",'value'),
                  State(f"{title}-title-input",'value')]
                 )
    def update_time_series(session_data,n_clicks,color_scheme,domain,title):
        return create_bar_chart(color_scheme,session_data,domain,title)

for title in title_list:
    update_bar_chart_callback(title)

###Callback for dropdown menu in the Notes page
@app.callback(Output('notes-dropdown','options'),
              Input('notes-domaindropdown', 'value'),
              )
def notes(domain,state=False):
    if domain is None:
        return notes_options
    else:
        notes_domain_options=metadata['Variable_Label'][metadata['Domain1']==domain].tolist()
        return notes_domain_options
    
@app.callback(Output('notes-content','children'),
              Input('notes-dropdown', 'value'),
              )
def notes(measure,state=False):
    source=metadata_full['Source'][metadata_full['Variable_Label']==measure]
    description=metadata_full['Data_Description'][metadata_full['Variable_Label']==measure]
    provenance=metadata_full['Data_Provenance'][metadata_full['Variable_Label']==measure]
    datalink=metadata_full['Data_link'][metadata_full['Variable_Label']==measure]
    metadatalink=metadata_full['Metadata_link'][metadata_full['Variable_Label']==measure]
    coverage=metadata_full['Coverage'][metadata_full['Variable_Label']==measure]
    frequency=metadata_full['Frequency'][metadata_full['Variable_Label']==measure]
    unit=metadata_full['Unit'][metadata_full['Variable_Label']==measure]
    geography=metadata_full['Geography'][metadata_full['Variable_Label']==measure]
    uncertainty=metadata_full['Uncertainty'][metadata_full['Variable_Label']==measure]
    citation=metadata_full['Citation'][metadata_full['Variable_Label']==measure]
    
    if measure is None:
        return html.Div('')
    else:
        return html.Div([
            html.I('Data source:'),html.P(source),
            html.I('Description of the data source:'),html.P(description),
            html.I('Provenance of the data source:'),html.P(provenance),
            html.I('Link to the data source:'),html.P(datalink),
            html.I('Link to metadata for the data source:'),html.P(metadatalink),
            html.I('Coverage of the data source:'),html.P(coverage),
            html.I('Frequency of releases for the data source:'),html.P(frequency),
            html.I('Unit of analysis for the data source:'),html.P(unit),
            html.I('Geography covered by the data source:'),html.P(geography),
            html.I('Measures of uncertainty in the data source:'),html.P(uncertainty),
            html.I('Citation for data source:'),html.P(citation)
    ])

###Start of Shannan's code
# Define callback to update the plot based on selected category, geo IDs, years, plot type, and color scheme
@app.callback(
    [Output('category-plot', 'figure'),
     Output('warning-message', 'children')],
    [Input('category-dropdown', 'value'),
     Input('year-dropdown', 'value'),
     Input('geo-id-dropdown', 'value'),
     Input('plot-type-radio', 'value'),
     Input('color-scheme-dropdown', 'value')]
)
def update_category_plot(selected_category, selected_years, selected_geo_ids, plot_type, color_scheme):
    try:
        filtered_vars = varcats[varcats['CATEGORY'] == selected_category]['VARIABLE']
        filtered_data = merged_data[merged_data['GEO_ID'].isin(selected_geo_ids) & merged_data['YEAR'].isin(selected_years)][['YEAR', 'GEO_ID', 'GEO_NAME'] + list(filtered_vars)]
        filtered_data = filtered_data.dropna(subset=filtered_vars)
        if filtered_data[filtered_vars].isnull().all().all():
            return {}, f"Warning: No data available for the selections. Data might be available for other geographies or years."
        
        melted_data = filtered_data.melt(id_vars=['YEAR', 'GEO_ID', 'GEO_NAME'], var_name='Variable', value_name='Value')
        
        # Sort by year and category in varcats
        variable_order = varcats[varcats['CATEGORY'] == selected_category]['VARIABLE'].tolist()
        melted_data['Variable'] = pd.Categorical(melted_data['Variable'], categories=variable_order, ordered=True)
        melted_data = melted_data.sort_values(["YEAR", "Variable"], ascending=[True, True])

        melted_data['Variable'] = melted_data['Variable'].map(column_label_map)  # Map variable names to labels from varcats
        ylabel = ylabel_map.get(selected_category, 'Value')  # Get Y-axis label from varcats

        # Check if one of the ratings variables
        for col in variable_order:
            if col in ratings_cols:
                melted_data['Value'] = pd.Categorical(melted_data['Value'], categories=ratings_vars, ordered=True)
                melted_data = melted_data.sort_values(['Value', 'YEAR', 'GEO_ID'])

        # Drop NA values
        melted_data = melted_data.dropna(subset=['Value'])
    
        # Check if sum of values of the variables in the selected category equals 1 for each bar with a tolerance of 0.3 (i.e., 99.7 and 100.3 are acceptable)
        numeric_vars = melted_data[melted_data['Value'].apply(lambda x: isinstance(x, (int, float)))]
        barmode = 'group'
        if not numeric_vars.empty:
            grouped_data = numeric_vars.groupby(['GEO_NAME', 'YEAR', 'Variable']).sum().reset_index()
            sum_check = grouped_data.groupby(['GEO_NAME', 'YEAR'])['Value'].sum().reset_index()
            sum_check['Value'].replace(0, 100, inplace=True)
            barmode = 'stack' if sum_check['Value'].apply(lambda x: np.isclose(x, 100, atol=.3)).all() else 'group'

        # Get the colors for the selected color scheme
        colors = color_schemes.get(color_scheme, px.colors.qualitative.Plotly)

        if plot_type == 'time_series':
            if len(filtered_vars)==1:
                fig = px.line(melted_data, x='YEAR', y='Value',
                              color='GEO_NAME', markers=True,
                              labels={'Value': ylabel, 'YEAR': 'Year', 'GEO_NAME': 'Geography'},
                              color_discrete_sequence=colors)
            else:
                fig = px.line(melted_data, x='YEAR', y='Value',
                              facet_col='GEO_NAME', color='Variable',
                              markers=True, labels={'Value': ylabel, 'YEAR': 'Year', 'GEO_NAME': 'Geography'},
                              color_discrete_sequence=colors)
            fig.update_xaxes(tickmode='array', tickvals=sorted(selected_years))
            fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        else:
            if len(filtered_vars)==1:
                fig = px.bar(melted_data, x='GEO_NAME', y='Value', color='GEO_NAME', barmode=barmode, facet_col='YEAR', labels={'Value': ylabel, 'YEAR': 'Year', 'GEO_NAME': 'Geography'}, color_discrete_sequence=colors)
            else:
                fig = px.bar(melted_data, x='GEO_NAME', y='Value', color='Variable', barmode=barmode, facet_col='YEAR', labels={'Value': ylabel, 'YEAR': 'Year', 'GEO_NAME': 'Geography'}, color_discrete_sequence=colors)
            fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
            

        fig.update_layout(title=f'{selected_category} for Selected Geographies and Years')
        return fig, ""
    except Exception as e:
        return {}, f"An error occurred while generating the plot: {str(e)}"

# Define callback to update the table based on selected category, geo IDs, and years
@app.callback(
    Output('comparison-table', 'children'),
    [Input('category-dropdown', 'value'),
     Input('year-dropdown', 'value'),
     Input('geo-id-dropdown', 'value')]
)
def update_comparison_table(selected_category, selected_years, selected_geo_ids):
    try:
        filtered_vars = varcats[varcats['CATEGORY'] == selected_category]['VARIABLE']
        filtered_data = merged_data[merged_data['GEO_ID'].isin(selected_geo_ids) & merged_data['YEAR'].isin(selected_years)][['YEAR', 'GEO_ID', 'GEO_NAME'] + list(filtered_vars)]

        if filtered_data[filtered_vars].isnull().all().all():
            return html.P(f"Warning: No data available for the selected geographies or years.")
        
        filtered_data = filtered_data.dropna(subset=filtered_vars)
        filtered_data.sort_values( by=['GEO_ID','YEAR'] , inplace=True )
        
        # Format numerical values with commas
        for col in filtered_data.columns:
            if col not in ['GEO_NAME', 'GEO_ID', 'YEAR'] and filtered_data[col].dtype in ['float64', 'int64']:
                filtered_data[col] = filtered_data[col].apply(lambda x: "{:,.0f}".format(x) if pd.notna(x) and x == int(x) else "{:,.2f}".format(x) if pd.notna(x) else np.nan)

        table = dash_table.DataTable(
            columns=[{"name": column_label_map.get(i, i), "id": i} for i in filtered_data.columns],  # Map columns to readable labels
            data=filtered_data.to_dict('records'),
            sort_action='native', #Enables column sorting
            style_table={'overflowX': 'auto', 'maxWidth': '95%'},  # Adjust the maxWidth as needed
            style_cell={
                'whiteSpace': 'normal',
                'height': 'auto',
                'textAlign': 'center',  # Center align the content
                'fontSize': 14
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold',
                'textAlign': 'center'  # Center align the header
            },
            export_format='csv',  #To downlaod data table as a csv file
            export_headers='display',  #Keep formatting for csv file as it displays on screen
            style_data_conditional=[
                {
                    'if': {
                        'filter_query': '{Value} contains "-"',
                        'column_id': 'Value',
                    },
                    'textAlign': 'left',
                },
                {
                    'if': {
                        'filter_query': '{Value} contains "."',
                        'column_id': 'Value',
                    },
                    'textAlign': 'right',
                },
                {
                    'if': {
                        'filter_query': '{Value} contains ","',
                        'column_id': 'Value',
                    },
                    'textAlign': 'right',
                },
            ],
            # Format number columns
            style_data={
                'if': {'column_type': 'numeric'},
                'textAlign': 'right',
            },
        )

        return table
    except Exception as e:
        return html.P(f"An error occurred while generating the table: {str(e)}")
    
shannan_viz=html.Div(
    [
        dbc.Row([html.P("")]),
        dbc.Row([
            dbc.Col(
                [html.I("Select Chart Type:"),dcc.RadioItems(id='plot-type-radio',
                                                                options=[
                                                                    {'label': ' Time Series', 'value': 'time_series'},
                                                                    {'label': ' Bar Chart', 'value': 'barplots'}
                                                                ],
                                                                value='time_series')
                ],
            ),
            dbc.Col(
                [html.I("Select Color Scheme for Color Blindness:"),dcc.Dropdown(id='color-scheme-dropdown',
                                                                                 options=[
                                                                                     {'label': 'Default', 'value': 'default'},
                                                                                     {'label': 'Monochrome (Black & White)', 'value': 'monochrome'},
                                                                                     {'label': 'Deuteranopia (Issues with Green)', 'value': 'deuteranopia'},
                                                                                     {'label': 'Protanopia (Issues with Red)', 'value': 'protanopia'},
                                                                                     {'label': 'Tritanopia (Issues with Blue)', 'value': 'tritanopia'}
                                                                                 ],
                                                                                 value='default'
                                                                                ),
                ]
            ),
        ]),
        #Added note to viewers about graph and data table display updates when no data is available
        html.Div("Note for viewers: All available data will display for your selections",
                 style={'position': 'absolute', 'top': 70, 'right': 10, 'fontSize': 20,  'color': 'black'}),
        dbc.Row([html.Div("")]),
        #These are the dropdown slections, inputs determined above, get domain uses a function to determine
        dbc.Row([
            dbc.Col(
                [html.I("Select Measure:"),dcc.Dropdown(id='category-dropdown',
                                                        options=[{'label': category, 'value': category} for category in categories],
                                                        value=categories[0])
                ]
            ),dbc.Col(
                [html.I("Select Year(s):"),dcc.Dropdown(id='year-dropdown',
                                                        options=[{'label': year, 'value': year} for year in sorted(years)],
                                                        value=sorted(list(years)),
                                                        multi=True)
                ]
            ),
            dbc.Col(
                [html.I("Select Geographic Area(s):"),dcc.Dropdown(id='geo-id-dropdown',
                                                                             options=[{'label': row['combined'], 'value': row['GEO_ID']} for _, row in geo_name_id.iterrows()],
                                                                             value=[geo_ids[0]],
                                                                             multi=True)
                ]
            )
        ]),
        # Warning message
        html.Div(id='warning-message', style={'color': 'red', 'fontWeight': 'bold'}),
        # Time series or barplot
        dcc.Graph(id='category-plot'),
        # Table to display values
        html.Div(id='comparison-table', style={'textAlign': 'center'}),
        # Center the table
    ]
)

#Find My Community Tool
find_community_tool = html.Div(
    [
        html.H4("STEP 1: Find my Community",style={"color": "#CF5008"}),
        html.P("Data is available for your region, division, state, county, tribal area, and Census \
        tract (which is a small county subdivision used to represent neighborhoods)."),
        html.I("Instructions:"),
        html.Br(),
        html.I("1. Enter your address in the fields below. All fields must be filled in."),
        html.Br(),
        html.I("2. Push the Submit button to enter."),
        html.Br(),
        html.I("3. The webpage will refresh afterwards."),
        html.Br(),
        html.Br(),
        dbc.Row(
            [#Below are inputs for user to add address, city, state, zip in order to use census API and populate other areas
                dbc.Col(
                    [html.Label("Street Address:"),
                     dcc.Input(id="address_input",
                               type="text",
                               placeholder="",
                               value="")
                    ],
                ),
                dbc.Col(
                    [html.Label("City:"),
                     dcc.Input(id="city_input",
                               type="text",
                               placeholder="",
                               value=""
                              )
                    ]
                )
            ]
        ),       
        dbc.Row(
         [
             dbc.Col(
                 [
                     html.Label("State:"),
                     dcc.Input(id="state_input",
                               type="text",
                               placeholder="",
                               value=""
                              ),
                 ]
             ),
             dbc.Col(
                 [
                     html.Label("Zip Code:"),
                     dcc.Input(id="zipcode_input",
                            type="text",
                               placeholder="",
                               value=""
                              ),
                 ],
             )
         ]
        ),
        dbc.Row([dbc.Col(html.Button('Submit',id='button-1'),width=2)]),
        #dbc.Card([html.Div(id='API_output')], className="border-0 bg-transparent"),
        html.Br(),
        html.I("4. Once you have entered an address and pushed the Submit button above, a menu of \
        different geography levels will appear below."),
        html.Br(),
        html.I("5. Select the geography level you are interested in."),
        html.Br(),
        html.I("6. The webpage will refresh and update the cards in Step 2 below."),    
        html.Br(),  
        html.Br(),
        dcc.RadioItems(id="georadioitems"),
        html.Br(),
        html.I("7. You will know the webpage has refreshed when your selection appears under Step 2.\
        Note that even though the address fields above will be blank after you make your selection,\
        you can still change your selection at any time."),    
        #store output_dict to get GEO_IDs
        dcc.Store(id='output_dict'),
        ]    
)

#original API tool, no longer in use with updates, leaving in case I need to refernce again
"""
#Callbacks for find my community tool
@app.callback(Output("API_output",'children'),
              inputs=Input('button-1', 'n_clicks'),
              state=[State('address_input','value'),
              State('city_input','value'),
              State("state_input",'value'),
              State("zipcode_input",'value')],
             )
#API Find my community tool function
##List of layers for reference: https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Current/MapServer
##API documenation: https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf

def adress_geo_finder(n_clicks,address,city,state,zip_code):
    address_concat = address +","+city+","+state+","+zip_code
    base_url = 'https://geocoding.geo.census.gov/geocoder'
    return_type = '/geographies'
    search_type = '/onelineaddress'
    url = base_url + return_type + search_type
    params = {
        'address': address_concat,
        'benchmark': 'Public_AR_Current',
        'format': 'json',
        'vintage':'Current_Current',
        'layers':'all'
    }

    response = requests.get(url,params=params)

    result = response.json()['result']['addressMatches'][0]['geographies']
    
    
    new_dict ={}
    
    
    for i in result:
        if i == 'States':
            new_dict['State'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
        elif i == 'Census Regions':
            new_dict['Census Region'] =(result[i][0]['NAME'],"R"+result[i][0]['GEOID'])
        elif i == 'Census Divisions':
            new_dict['Census Division'] =(result[i][0]['NAME'],"D"+result[i][0]['GEOID'])
        elif i == 'Counties':
            new_dict['County'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
        elif i == 'Census Tracts':
            new_dict['Census Tract'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
        #elif i == 'Census Block Groups':
            #new_dict['Census Block Group'] =(result[i][0]['NAME'],"D"+result[i][0]['GEOID']
        elif i in ('Hawaiian Home Lands',
                   'Oklahoma Tribal Statistical Areas',
                   'Alaska Native Village Statistical Areas',
                   'Tribal Designated Statistical Areas',
                   'State Designated Tribal Statistical Areas',
                   'Federal American Indian Reservations'):
            new_dict['Tribal Statistical Area'] =(result[i][0]['NAME'],result[i][0]['GEOID'])                            
    
    lines = []
    for key in ['Census Region','Census Division','State','County','Census Tract','Tribal Statistical Area']:
        if key in new_dict:
            line = f"My {key} is \"{new_dict[key][0]}\" and the ID for it is \"{new_dict[key][1]}\"."
            lines.append(line)
            
    output = [html.Div(line) for line in lines]
           
    return output
"""

#used for debuging, leaving in case I need to reference again
"""
@app.callback(
    Output('address-store-contents', 'children'),
    [Input('address-store', 'children')]
)
def print_address_store_contents(store_data):
    print("Address Store Contents:")
    print(store_data)
    return store_data
"""

#Stores a dictionary to populate the geo radio buttons, the georadio buttons are being stored.
@app.callback(
    Output('address-store', 'children'),
    [Input('button-1', 'n_clicks')],
    [State('address_input', 'value'),
     State('city_input', 'value'),
     State('state_input', 'value'),
     State('zipcode_input', 'value')]
)
def store_input(n_clicks, address, city, state, zip_code):
    if n_clicks and address and city and state and zip_code:
        return {'address': address, 'city': city, 'state': state, 'zipcode': zip_code}
    else:
        return dash.no_update

    
#Callbacks for radio button menu resulting from API Find my community tool

@app.callback([Output('georadioitems', 'options'),
               Output('output_dict', 'children')],
              [Input('button-1', 'n_clicks'),
               Input('address-store', 'children')]
             )
#Function to populate geography radio button menu
#API Find my community tool function
##List of layers for reference: https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_Current/MapServer
##API documenation: https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
def update_georadioitem_options(n_clicks,address_data):
    #this section use API to return data based on user address inputs
    address = address_data['address']
    city = address_data['city']
    state = address_data['state']
    zip_code = address_data['zipcode']

    address_concat = address +","+city+","+state+","+zip_code
    base_url = 'https://geocoding.geo.census.gov/geocoder'
    return_type = '/geographies'
    search_type = '/onelineaddress'
    url = base_url + return_type + search_type
    params = {
        'address': address_concat,
        'benchmark': 'Public_AR_Current',
        'format': 'json',
        'vintage':'Current_Current',
        'layers':'all'
    }

    response = requests.get(url,params=params)
    result = response.json()['result']['addressMatches'][0]['geographies']   
    
    #Creates a dictionary to store information
    new_dict ={}
    new_dict['Nation'] = ('Nation','us')
    for i in result:
        if i == 'States':
            new_dict['State'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
        elif i == 'Census Regions':
            new_dict['Census Region'] =(result[i][0]['NAME'],"R"+result[i][0]['GEOID'])
        elif i == 'Census Divisions':
            new_dict['Census Division'] =(result[i][0]['NAME'],"D"+result[i][0]['GEOID'])
        elif i == 'Counties':
            new_dict['County'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
        elif i == 'Census Tracts':
            new_dict['Census Tract'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
        #elif i == 'Census Block Groups':
            #new_dict['Census Block Group'] =(result[i][0]['NAME'],"D"+result[i][0]['GEOID']
        elif i in ('Hawaiian Home Lands',
                   'Oklahoma Tribal Statistical Areas',
                   'Alaska Native Village Statistical Areas',
                   'Tribal Designated Statistical Areas',
                   'State Designated Tribal Statistical Areas',
                   'Federal American Indian Reservations'):
            new_dict['Tribal Statistical Area'] =(result[i][0]['NAME'],result[i][0]['GEOID'])
    #These are labels for the push buttons
    if len(new_dict)==7:
        keys=['Tribal Statistical Area','Census Tract','County','Census Region','Census Division','State']
        radioitem_dict={key:f" My {key}: {new_dict[key][0]} (Code: {new_dict[key][1]})" for key in keys}
    else:
        keys=['Census Tract','County','Census Region','Census Division','State']
        radioitem_dict={key:f" My {key}: {new_dict[key][0]} (Code: {new_dict[key][1]})" for key in keys}
    radioitem_dict['Nation']=' My Country: United States (Code: 0)'
    
    #was having trouble with outputting dictionary, solution was to output as a json and load the json back in
    #unsure if there is a better method
    output_json = json.dumps(new_dict)

    #returns button options and json dictionary for reference
    return radioitem_dict,output_json

#Callback to store session geography selection as data table for use to populate cards
@app.callback(
    [Output('session_data', 'data'),
     Output('community-title','children')],
    [Input('georadioitems', 'value')],
    [State('output_dict', 'children')]
)
#Function to create session based on geography selection
def filter_geo(georadioitems,output_json):
    
    #loads back in the json from update_georadioitem_options
    output_dict = json.loads(output_json)
    
    #gets code, using user selection from push buttons and dictionary
    code = output_dict[georadioitems][1]
    
    #filters larger data set to pull rows where code matches
    filtered_df=df_update[df_update['GEO_ID']==code]
    
    # createstext to populate webpage so users know they selected
    community_text = f"My "+georadioitems+": "+output_dict[georadioitems][0]
    
    #returns dataframe as dictionary for session data and text which goes in web layout
    return filtered_df.to_dict('records'), community_text


#This was used for testing and debuging 
""" #Callback to store session geography selection
@app.callback(Output('result','children'),
              Input('session_data', 'data'),
              )

def testfunction(data):
    print(data[0]['TOTAL_POPULATION'])
    value=data[0]['TOTAL_POPULATION']
    return value
    print("test")
"""

#Content
##Style arguments
###Positions content to the right of the sidebar and adds padding
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

##Layout
###Individual domains
content = html.Div(id="page-content", style=CONTENT_STYLE)


####Callbacks to provide domain-specific content to the webpage
@app.callback(Output("page-content", "children"),
              [Input("url", "pathname"),
               Input('session_data','data')],
              State('community-title', 'children')
             )

####Define function to get content
#####The following are the same for all domains: heading, a line break, the Find My Community Tool, and another line break
#####The following are domain-specific: cards and accordion
def render_page_content(pathname,session_data=None,community_title=None):
    if pathname == "/":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community in general."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("General",session_data),
                compile_accordion("General",session_data)
            ],
        )
    elif pathname == "/housing_infrastructure_transportation":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's housing, infrastructure, and transportation."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Infrastructure, Housing, & Transportation",session_data),
                compile_accordion("Inrastructure, Housing, & Transportation",session_data)
            ],
        )
    elif pathname == "/health_nutrition":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's health and nutrition."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Health & Nutrition",session_data),
                compile_accordion("Health & Nutrition",session_data)
            ],
        )
    elif pathname == "/education":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's education."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Education",session_data),
                compile_accordion("Education",session_data)
            ],
        )
    elif pathname == "/income_poverty_socialservices":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's income, poverty, and social services."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Income, Poverty, & Social Services",session_data),
                compile_accordion("Income, Poverty, & Social Services",session_data)
            ],
        )
    elif pathname == "/publicsafety":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's public safety."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Public Safety",session_data),
                compile_accordion("Public Safety",session_data)
            ],
        )
    elif pathname == "/economicvitality":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's economic vitality."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Economic Vitality",session_data,session_data),
                compile_accordion("Economic Vitality")
            ],
        )
    elif pathname == "/environment_naturalresources":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's environment and natural resources."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Environmnet & Natural Resources",session_data),
                compile_accordion("Environmnet & Natural Resources",session_data)
            ],
        )
    elif pathname == "/disasterprevention_relief":
        return html.Div(
            [
                heading,html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                html.I('Instructions:'),
                html.Br(),
                html.I("1. The below cards share data about your community's disaster prevention and relief."),
                html.Br(),
                html.I("2. Click on the links in the cards to see detailed graphs."),
                html.Br(),
                html.I("3. To see cards about a different topic, use the sidebar on the left."),
                dbc.Row([html.P("")]),
                compile_cards("Disaster Prevention and Relief",session_data),
                compile_accordion("Disaster Prevention and Relief",session_data)
            ],
        )
    elif pathname == "/compare":
        return html.Div(
            [
                html.H1("Census Data Graph Customizer for Grant Writers"),
                html.Hr(),find_community_tool,html.Hr(),
                html.H4("STEP 2: Get data about my community",style={"color": "#CF5008"}),
                html.H2(community_title),
                shannan_viz
            ]
        )          
    elif pathname == "/notes":
        return html.Div(
            [
                heading,
                html.Hr(),
                html.H4("NOTES ON THIS APP"),
                html.P('The Census Data Explorer for Grant Writers is an experimental tool tailored toward users \
                looking for data to complete grant application. Data are provided from the U.S. Census Bureau, \
                U.S. Department of Agriculture, Center for Disease Control and Prevention, Federal Emergency \
                Management Agency, National Oceanic and Atmosphere Administration, U.S. Energy Information \
                Administration, Bureau of Economic Analysis, University of Michigan Institute for Social Research, \
                and Feeding America for years 2015 through 2024, where applicable.'),
                html.Hr(),
                html.H4("NOTES ON DATA SOURCES"),
                html.P('Estimates displayed in this application come from many data sources. To get more information\
                about where a specific estimate came from, use the below dropdown menu to select an estimate.'),
                dbc.Row([
                    dbc.Col(html.I('Select a topic (optional)')),
                    dbc.Col(html.I('Select an estimate'))
                ]),
                dbc.Row([
                    dbc.Col(dcc.Dropdown(options=domain_options,placeholder='Select a topic',id='notes-domaindropdown')),
                    dbc.Col(dcc.Dropdown(options=notes_options,placeholder='Select an estimate',id='notes-dropdown'))
                ]),
                html.Br(),
                html.Div(id='notes-content'),
                html.Hr(),
                html.H4("ADDITIONAL RESOURCES"),
                html.A("U.S. Census Bureau: Our Surveys & Programs",
                       href="https://www.census.gov/programs-surveys.html",
                       target="_blank",style={"color": "blue", "text-decoration": "underline"}),
                html.Br(),
                html.A("U.S. Census Bureau: Census Survey Explorer (beta)",
                       href="https://ask.census.gov/support/case",
                       target="_blank",style={"color": "blue", "text-decoration": "underline"})
            ]
        )
    #If the user tries to reach a different page, return a 404 message
    return html.Div(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognized."),
        ],
        className="p-3 bg-light rounded-3",
    )


###Overall
app.layout = html.Div(
    [
        ####Represents the location or address bar
        dcc.Location(id="url"),
        ####Session store will take the initial data the first time the page is loaded and then lose it when the browser/tab closes.
        dcc.Store(id="session_output",storage_type='session'),
        #these store some of the stuff for populating the cards
        dcc.Store(id='address-store'),
        dcc.Store(id='session_data', data={}),
        dcc.Store(id='georadioitems'),
        dcc.Store(id='community-title'),
        ####Sidebar (see above)
        sidebar,
        ####Content (see above)
        content,
    ]
)

#Create data tool
if __name__ == '__main__':
    app.run_server(jupyter_mode='external', port=8051)

Dash app running on http://127.0.0.1:8051/


[2025-04-01 11:18:45,153] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Colin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Colin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Colin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Colin\AppData\Local\Packages\PythonSoftwareFoundation